## Imports 

In [1]:
import numpy as np
import pandas as pd 
from collections import Counter
from statsmodels.distributions.empirical_distribution import ECDF
import os
import glob
import copy

## Opening the CSV files 

In [2]:
dataframes = [pd.read_csv(file, sep=',', index_col=0) for file in sorted(glob.glob('../preprocessed_datasets' + "/*."+'csv'))]
cohorts = [file.strip(".csv") for file in sorted(os.listdir('../preprocessed_datasets'))]

In [3]:
# reduce to BL visit
all_cohorts = dict()
for name, df in zip(cohorts, dataframes):
    all_cohorts[name] = df.loc[df["Visit"] == 1]
    
#all_cohorts_ctl = copy.deepcopy(all_cohorts)
all_cohorts_ctl = dict()
for name, df in zip(cohorts, dataframes):
    all_cohorts_ctl[name] = df.loc[(df["Visit"] == 1) & (df["Diagnosis"].astype(str) == 'CU')]   

## Functions to perform essential calculations 

In [4]:
def cat_stat_df(dfs, result):
    """Counting different categories, calculate the % of categorical features, store results in a df"""
    
    categorical = {'APOE4': [2.0, 1.0], 'Sex': ['Female'], 'Diagnosis': ['CU', 'MCI', 'AD']}
    column_cat = ['Sex', 'Diagnosis', 'APOE4']

    for cohort in dfs:
        
        if dfs[cohort].empty==False:
            calc_dict = dict()
            df = dfs[cohort]

            for col in column_cat:
                ca = Counter(df[col].dropna())
                calc_dict[col] = ca

            cohort_df = pd.DataFrame(calc_dict).transpose()
            cohort_df = cohort_df.dropna(how='all')
            cohort_df.loc[cohort] = cohort_df.sum()
           
            for i in categorical:
                
                if i == 'Diagnosis':
                    
                    if i in cohort_df.index: 
                        result.loc[cohort, categorical[i]] = cohort_df.loc[cohort, cohort_df.loc[i].notna()].astype(int)
                        result.loc[cohort, categorical[i]] = result.loc[cohort, categorical[i]].replace({np.nan: 0})
                        result.loc[cohort, 'n'] = int(sum(cohort_df.loc[cohort, cohort_df.loc[i].notna()]))
                    
                    else:
                        result.loc[cohort, i] = np.nan
                        result.loc[cohort, 'n'] = int(len(dfs[cohort].index))
                
                elif i == 'APOE4':
                    
                    if 'APOE4' in list(cohort_df.index.astype(str)):
                        
                        if '2.0' not in list(cohort_df.columns.astype(str)) and '2' not in list(cohort_df.columns.astype(str)):
                            cohort_df[2.0] = np.nan
                        
                        result.loc[cohort, i] = round(100 * sum([val for val in cohort_df.loc[i, categorical[i]]]) / 
                                                     sum([val for val in cohort_df.loc[i].dropna()]), 1)
                    
                    else:
                        result.loc[cohort, i] = np.nan
                
                elif i == 'Sex':
                    
                    if (i in cohort_df.index) & ("Female" in cohort_df.columns):
                        result.loc[cohort, i] = round(100 * sum([val for val in cohort_df.loc[i, categorical[i]]]) 
                                                      / sum([val for val in cohort_df.loc[i].dropna()]), 1)
                    else:
                        result.loc[cohort, i] = 0
                        
        result.loc[cohort, 'Total'] = int(len(dfs[cohort].index))
    
                    
    result.rename(columns={"Sex": "Female %", "APOE4": "APOE4 %"}, inplace=True)
              
    return result

In [5]:
def num_quantiles(dfs, dfs_ctl, result):
    """Calculating std and mean and evalute quantiles under the ECDF for all cohorts"""
    
    column_num = ['Age', 'CDR', 'Education', 'MMSE', 'CDRSB', 'Hippocampus', 'A-beta', 'Ttau', 'Ptau']
    non_int_cols=["CDR", "CDRSB"]
    biomarker = ['Hippocampus', 'A-beta', 'Ttau', 'Ptau']
    
    for df, ctl_df in zip(dfs, dfs_ctl):
        
        dfn = dfs[df]
        dfn_ctl = dfs_ctl[ctl_df]
        calc_dict = dict()
        calc_dict_ctl = dict()
        
        for col in column_num:
            quants = []
            
            if (dfn.empty == False) & (col in dfn.columns)==True:
                
                if len(dfn.index.unique()) > 2:

                    # return nan if no data
                    if pd.isnull(dfn[col].quantile()):
                        calc_dict[col] = np.nan
                        continue

                    if col in non_int_cols:

                        for i in [.25, .5, .75]:
                            quants.append(round(dfn[col].quantile(i), 1))
                    else:

                        for i in [.25, .5, .75]:
                            quants.append(int(round(dfn[col].quantile(i), 0)))

                    # create and save string to return
                    calc_dict[col] =  str(quants[0]) + ', ' + str(quants[1]) + ', ' + str(quants[2])
                
                elif len(dfn.index.unique()) == 2:
                    
                    if col == 'Age':
                        quants = (list(dfn.iloc[0:][col].values))
                        calc_dict[col] = str(int(quants[0])) +  ', ' + str(int(quants[1]))
                     
                    else:
                        quants = (list(dfn.iloc[0:][col].values))
                        calc_dict[col] = str(round(quants[0], 1)) +  ', ' + str(round(quants[1], 1))
                
                else:
                    
                    if col == 'Age':
                        calc_dict[col] = int(dfn.iloc[0][col]) 
                    
                    else:
                        calc_dict[col] = round(dfn.iloc[0][col], 1)
                    

        for col in biomarker:
            if len(dfn_ctl.index.unique()) > 2:

                if (dfn_ctl.empty == False) & (col in dfn.columns)==True:

                    quants = []
                    ctl_dat = dfn_ctl[col].dropna()

                    # return nan if no data or no control distribution to compare to
                    if (pd.isnull(dfn[col].quantile())) or not (ctl_dat.any()):
                        calc_dict_ctl[col] = np.nan
                        continue

                    ecdf = ECDF(ctl_dat)

                    for i in [.25, .5, .75]:
                        value = int(round(ecdf(dfn[col].quantile(i)) * 100))
                        quants.append(value)

                    # create and save string to return
                    calc_dict_ctl[col] =  str(quants[0]) + ', ' + str(quants[1]) + ', ' + str(quants[2])
                
                else:
                    calc_dict_ctl[col] = np.nan
                

        for clin, bio in zip([calc_dict], [calc_dict_ctl]):
        
            for marker in biomarker:
                
                if (marker in clin) & (marker in bio): 

                    if pd.notnull(clin[marker]):
                        clin[marker] += " (" + str(bio[marker]) + ")"
                        
                else:
                    continue

        df_quan = pd.DataFrame(calc_dict, index=[df])
        
        for col in df_quan.columns:
            result.loc[df, col] = df_quan.loc[df, col]
            
    result.rename(columns={"Ttau": "tTau", "Ptau": "pTau"}, inplace=True)
        
    return result

## Make an empty dataframe to fill in with the results

In [6]:
results = pd.DataFrame(index = all_cohorts.keys(), columns = [col for col in all_cohorts['AIBL'].columns])
results.index.name = 'Name of Dataset'

for i in ['CU', 'MCI', 'AD', 'Total']:
    results[i] = np.nan

results.drop(columns=['Diagnosis', 'Visit'], inplace=True)

results = cat_stat_df(all_cohorts, results)
results = num_quantiles(all_cohorts, all_cohorts_ctl, results)

## Final table 

In [7]:
# sort columns
results = results[["n", "Total", "CU", "MCI", "AD", "Female %", "Age", "Education", "APOE4 %", 
                   "MMSE", "CDR", "CDRSB", "Hippocampus", "A-beta", "tTau", "pTau"]]

In [8]:
results

,n,Total,CU,MCI,AD,Female %,Age,Education,APOE4 %,MMSE,CDR,CDRSB,Hippocampus,A-beta,tTau,pTau
Name of Dataset,,,,,,,,,,,,,,,,
A4,6945.0,6945.0,6945.0,0.0,0.0,57.7,"68, 71, 75","14, 16, 18",34.3,"28, 29, 30","0.0, 0.0, 0.0","0.0, 0.0, 0.0","6, 7, 7 (25, 50, 75)",NaN,NaN,NaN
ABVIB,280.0,280.0,NaN,NaN,NaN,42.9,"73, 77, 82","14, 16, 18",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ADNI,2218.0,2249.0,813.0,1016.0,389.0,47.0,"68, 73, 78","14, 16, 18",45.6,"26, 28, 29","0.0, 0.5, 0.5","0.0, 1.0, 2.0","5948, 6864, 7651 (7, 26, 60)","596, 854, 1396 (12, 27, 57)","193, 258, 350 (36, 66, 88)","17, 24, 34 (37, 69, 91)"
AIBL,1118.0,1378.0,803.0,134.0,181.0,57.9,"67, 73, 79","10, 12, 15",36.0,"26, 28, 30","0.0, 0.0, 0.5","0.0, 0.0, 1.0","3, 3, 3 (11, 37, 71)","445, 567, 802 (nan)","238, 366, 516 (nan)","43, 64, 81 (nan)"
ANM,1702.0,1703.0,793.0,397.0,512.0,59.3,"71, 77, 81","8, 11, 14",38.8,"24, 28, 29","0.0, 0.5, 0.5","0.0, 0.5, 4.0","5311, 6270, 7142 (2, 20, 54)",NaN,NaN,NaN
ARWIBO,1965.0,2617.0,1476.0,208.0,281.0,60.8,"48, 64, 74","5, 8, 13",29.3,"24, 28, 29","0.0, 0.0, 0.5",NaN,"6242, 7415, 8266 (5, 30, 63)","331, 445, 667 (0, 0, 100)","232, 367, 602 (0, 0, 100)","45, 65, 93 (0, 0, 0)"
DOD-ADNI,208.0,458.0,181.0,27.0,0.0,0.9,"66, 68, 72","13, 15, 17",26.6,"27, 29, 30","0.0, 0.0, 0.5","0.0, 0.0, 0.5","7145, 7777, 8498 (24, 50, 75)","797, 1225, 1512 (27, 51, 78)","154, 198, 254 (26, 52, 78)","13, 17, 22 (26, 53, 78)"
EDSD,474.0,474.0,183.0,140.0,151.0,52.1,"67, 71, 76","10, 12, 14",47.0,"24, 27, 29",NaN,NaN,"5937, 6833, 7807 (2, 17, 55)","414, 534, 852 (nan)","241, 386, 609 (nan)","51, 72, 102 (nan)"
EMIF,1093.0,1199.0,366.0,526.0,201.0,45.9,"62, 68, 74","9, 12, 15",NaN,"25, 28, 29","0.5, 0.5, 0.5",NaN,"6338, 7201, 7981 (5, 26, 63)","383, 522, 732 (10, 37, 66)","160, 280, 506 (37, 74, 95)","37, 52, 74 (39, 71, 93)"


### Outputs

In [9]:
results[["Female %", "Age", "Education", "APOE4 %", "MMSE", "CDR", "CDRSB", "Hippocampus", 
        "A-beta", "tTau", "pTau"]].to_csv("../adata_resources/ALL_summary_stats.csv")

In [10]:
print("N all cohorts: ", results["Total"].sum())

N all cohorts:  66807.0
